In [1]:
import os
import io
from PIL import Image
import time
import shutil
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from selenium.webdriver.common.by import By

In [2]:
players = ['Larry_Bird']
nbr_imgs = 5

In [3]:
IMAGES_PATH = os.path.join('data', 'dream_team')

In [4]:
wd = webdriver.Chrome()

In [5]:
def download_image(download_path, url, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + "/" + file_name 
        
        with open(file_path, 'wb') as f:
            image.save(f, 'JPEG')
    
    except Exception as e:
        print('FAILED -', e)

In [6]:
def get_images_from_google_images(wd, delay, max_images, player):
    def scroll_down(wd):
        wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(delay)
    url = f"https://www.google.com/search?q={player}&sca_esv=557430156&tbm=isch&source=lnms&sa=X&ved=2ahUKEwj_lvuHjuGAAxWiVKQEHRmrDQwQ_AUoAXoECAMQAw&biw=1280&bih=602&dpr=1"
    wd.get(url)
    
    image_urls = set()
    
    while len(image_urls) < max_images:
        scroll_down(wd)
        
        thumbnails = wd.find_elements(By.CLASS_NAME ,"Q4LuWd")
        
        for img in thumbnails[len(image_urls): max_images]:
            try:
                img.click()
                time.sleep(delay)
            except:
                continue
                
            images = wd.find_elements(By.CLASS_NAME,"iPVvYb")
            
            for image in images :
                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))
                           
    return image_urls 
                            
    

In [7]:
def execute_download():
    for player in players:
    
        if not os.path.exists(IMAGES_PATH):
            if os.name == 'posix':
                !mkdir -p {IMAGES_PATH}
            if os.name == 'nt':
                !mkdir {IMAGES_PATH}

        path = os.path.join(IMAGES_PATH, player)

        if not os.path.exists(path):
            !mkdir {path}

        urls = get_images_from_google_images(wd, 2, nbr_imgs, player)
        for i, url in enumerate(urls):
                download_image(path, url, player + str(i) + ".jpeg" )
    wd.quit()

In [8]:
output_folder = os.path.join('data')
percentage_to_move = 80

def move_even_number_of_files(source_folder, dest_folder, percentage_to_keep):
    all_files = os.listdir(source_folder)
    all_files.sort()  # Triez les noms de fichiers par ordre alphabétique
    
    num_files = len(all_files)
    num_files_to_move = int(num_files * percentage_to_move / 100)
    
    # Assurez-vous que num_files_to_keep est un nombre pair
    if num_files_to_move % 2 != 0:
        num_files_to_move -= 1
    
    sorted_files = all_files[:num_files_to_move]
    
    for file_name in sorted_files:
        source_path = os.path.join(source_folder, file_name)
        dest_path = os.path.join(dest_folder, file_name)
        shutil.move(source_path, dest_path)


In [9]:
def execute_lebelIMG():
    LABELIMG_PATH = os.path.join('data', 'labelImg_resources')

    if not os.path.exists(LABELIMG_PATH):
        !mkdir (LABELIMG_PATH)
        !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}
        
    if os.name == 'posix':
        !cd {LABELIMG_PATH} && make qt5py3
    if os.name == 'nt':
        !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc
    
    !cd {LABELIMG_PATH}  && python labelImg.py

In [10]:
execute_download()

In [11]:
execute_lebelIMG()

pyrcc5 -o libs/resources.py resources.qrc
^C


In [12]:
# Créer le dossier "train" au niveau du dossier parent
train_folder = os.path.join(IMAGES_PATH, "train")
os.makedirs(train_folder, exist_ok=True)

# Parcourir tous les sous-dossiers du dossier parent
for subfolder in os.listdir(IMAGES_PATH):
    subfolder_path = os.path.join(IMAGES_PATH, subfolder)
    
    # S'assurer que subfolder_path est un dossier
    if os.path.isdir(subfolder_path):
        move_even_number_of_files(subfolder_path, train_folder, percentage_to_move)